In [1]:
#import all the necessary packages

import pandas as pd
import random

import nltk
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk import stem
stemmer = stem.PorterStemmer()
from nltk import word_tokenize
nltk.download('stopwords')
from nltk.corpus import stopwords
stops = set(stopwords.words('english'))
import string
punct = list(string.punctuation)
from collections import Counter
import requests
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt
import numpy as np
import praw
import datetime
import json
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aoife\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import PyPDF2

In [3]:
# open the PDF file
policy = open(r'C:\Users\aoife\Documents\Capstone\Capstone Quant Policies\Northeastern University London Disability.pdf', 'rb')

# create a PDF reader object
pdf_reader = PyPDF2.PdfReader(policy)

In [4]:
# read the text from each page
text = ''
for page_num in range(len(pdf_reader.pages)):
    page = pdf_reader.pages[page_num]
    text += page.extract_text()

# close the PDF file
policy.close()

In [5]:
# split the text into sentences
sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

# create a dataframe with the sentences as rows
df = pd.DataFrame({'Sentence': sentences})

In [6]:
df

,Sentence
0,Student Disability Policy\nLast modi ed on Dec...
1,Students with the relevant suppor ng\ndocument...
2,"1.For the purposes of the Equality Act 2010, a..."
3,Disability in this context can include\na wide...
4,"Typically, this would be one or a combina on o..."
...,...
114,Note:\nStudent: What should the scribe do if t...
115,Should the scribe ask you to repeat it there\n...
116,Note:\nStudent: How will the scribe indicate t...
117,Note:\nStudent: Do you want the scribe to read...


In [7]:
#check to see if any rows are empty
rows_with_nan = df[df.isna().any(axis=1)]

len(rows_with_nan)

0

In [8]:
#check the length of the dataframe
len(df)

119

In [9]:
# convert each row in the dataframe to a string and then convert the datadrame to a list of strings
p1 = df.apply(lambda row: ' '.join(row.astype(str)), axis=1).tolist()

In [10]:
p1

['Student Disability Policy\nLast modi\x00ed on December 20th, 2023 at 1:36 pm\nIntroduc\x00on\nDe\x00ni\x00on of Disability\nResponsibili\x00es1.Northeastern University London (the University) posi\x00vely welcomes applica\x00ons from disabled students and is commi\x00ed to\nensuring every e\x00ort is made to allow all students to fully experience life at the University.',
 'Students with the relevant suppor\x00ng\ndocumenta\x00on (e.g. medical notes and/or reports from educa\x00onal psychologists) are eligible to apply for appropriate support\nwithout compromising academic standards.',
 '1.For the purposes of the Equality Act 2010, a person has a disability if they have a physical or mental impairment which has a\nsubstan\x00al and long-term adverse e\x00ect on their ability to carry out normal day-to-day ac\x00vi\x00es.',
 'Disability in this context can include\na wide range of issues and condi\x00ons.',
 'Typically, this would be one or a combina\x00on of the following categories:

In [11]:
# define a function to remove punctuation and numbers from a string
def remove_punctuation_and_numbers(text):
    # define a regular expression pattern to match punctuation characters and numbers
    pattern = r'[^A-Za-z\s]'
    # replace punctuation characters and numbers with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

# apply the previously created function to each string in the list
cleaned_p1 = [remove_punctuation_and_numbers(string) for string in p1]

# display the cleaned list of strings
print(cleaned_p1)

['Student Disability Policy\nLast modied on December th  at  pm\nIntroducon\nDenion of Disability\nResponsibiliesNortheastern University London the University posively welcomes applicaons from disabled students and is commied to\nensuring every eort is made to allow all students to fully experience life at the University', 'Students with the relevant supporng\ndocumentaon eg medical notes andor reports from educaonal psychologists are eligible to apply for appropriate support\nwithout compromising academic standards', 'For the purposes of the Equality Act  a person has a disability if they have a physical or mental impairment which has a\nsubstanal and longterm adverse eect on their ability to carry out normal daytoday acvies', 'Disability in this context can include\na wide range of issues and condions', 'Typically this would be one or a combinaon of the following categories\nSpecic learning diculty SpLD eg dyslexia\nVisual impairment  paral sight or blind\nHearing loss  paral hearing

In [12]:
# tokenize each string in the list of strings
tokenized_p1 = [word_tokenize(string) for string in cleaned_p1]

# display the tokenized list of strings
print(tokenized_p1)

[['Student', 'Disability', 'Policy', 'Last', 'modied', 'on', 'December', 'th', 'at', 'pm', 'Introducon', 'Denion', 'of', 'Disability', 'ResponsibiliesNortheastern', 'University', 'London', 'the', 'University', 'posively', 'welcomes', 'applicaons', 'from', 'disabled', 'students', 'and', 'is', 'commied', 'to', 'ensuring', 'every', 'eort', 'is', 'made', 'to', 'allow', 'all', 'students', 'to', 'fully', 'experience', 'life', 'at', 'the', 'University'], ['Students', 'with', 'the', 'relevant', 'supporng', 'documentaon', 'eg', 'medical', 'notes', 'andor', 'reports', 'from', 'educaonal', 'psychologists', 'are', 'eligible', 'to', 'apply', 'for', 'appropriate', 'support', 'without', 'compromising', 'academic', 'standards'], ['For', 'the', 'purposes', 'of', 'the', 'Equality', 'Act', 'a', 'person', 'has', 'a', 'disability', 'if', 'they', 'have', 'a', 'physical', 'or', 'mental', 'impairment', 'which', 'has', 'a', 'substanal', 'and', 'longterm', 'adverse', 'eect', 'on', 'their', 'ability', 'to', 'car

In [13]:
# as some strings are now empty, remove empty strings from a list of strings
cleaned_list_of_strings = [string for string in tokenized_p1 if string]

# and just in case that doesn't work,, remove empty strings from a list of lists of strings
cleaned_tokenized_p1 = [[token for token in tokens if token] for tokens in tokenized_p1]

In [14]:
cleaned_tokenized_p1

[['Student',
  'Disability',
  'Policy',
  'Last',
  'modied',
  'on',
  'December',
  'th',
  'at',
  'pm',
  'Introducon',
  'Denion',
  'of',
  'Disability',
  'ResponsibiliesNortheastern',
  'University',
  'London',
  'the',
  'University',
  'posively',
  'welcomes',
  'applicaons',
  'from',
  'disabled',
  'students',
  'and',
  'is',
  'commied',
  'to',
  'ensuring',
  'every',
  'eort',
  'is',
  'made',
  'to',
  'allow',
  'all',
  'students',
  'to',
  'fully',
  'experience',
  'life',
  'at',
  'the',
  'University'],
 ['Students',
  'with',
  'the',
  'relevant',
  'supporng',
  'documentaon',
  'eg',
  'medical',
  'notes',
  'andor',
  'reports',
  'from',
  'educaonal',
  'psychologists',
  'are',
  'eligible',
  'to',
  'apply',
  'for',
  'appropriate',
  'support',
  'without',
  'compromising',
  'academic',
  'standards'],
 ['For',
  'the',
  'purposes',
  'of',
  'the',
  'Equality',
  'Act',
  'a',
  'person',
  'has',
  'a',
  'disability',
  'if',
  'they',

In [15]:
#lemmatize
postlemma = []

for i in cleaned_tokenized_p1: 
    postlemma.append([lemmatizer.lemmatize(j) for j in i])

In [16]:
postlemma

[['Student',
  'Disability',
  'Policy',
  'Last',
  'modied',
  'on',
  'December',
  'th',
  'at',
  'pm',
  'Introducon',
  'Denion',
  'of',
  'Disability',
  'ResponsibiliesNortheastern',
  'University',
  'London',
  'the',
  'University',
  'posively',
  'welcome',
  'applicaons',
  'from',
  'disabled',
  'student',
  'and',
  'is',
  'commied',
  'to',
  'ensuring',
  'every',
  'eort',
  'is',
  'made',
  'to',
  'allow',
  'all',
  'student',
  'to',
  'fully',
  'experience',
  'life',
  'at',
  'the',
  'University'],
 ['Students',
  'with',
  'the',
  'relevant',
  'supporng',
  'documentaon',
  'eg',
  'medical',
  'note',
  'andor',
  'report',
  'from',
  'educaonal',
  'psychologist',
  'are',
  'eligible',
  'to',
  'apply',
  'for',
  'appropriate',
  'support',
  'without',
  'compromising',
  'academic',
  'standard'],
 ['For',
  'the',
  'purpose',
  'of',
  'the',
  'Equality',
  'Act',
  'a',
  'person',
  'ha',
  'a',
  'disability',
  'if',
  'they',
  'have'

In [17]:
#lower all cases
postfin1 = [[j.lower() for j in i] for i in postlemma]

In [18]:
postfin1

[['student',
  'disability',
  'policy',
  'last',
  'modied',
  'on',
  'december',
  'th',
  'at',
  'pm',
  'introducon',
  'denion',
  'of',
  'disability',
  'responsibiliesnortheastern',
  'university',
  'london',
  'the',
  'university',
  'posively',
  'welcome',
  'applicaons',
  'from',
  'disabled',
  'student',
  'and',
  'is',
  'commied',
  'to',
  'ensuring',
  'every',
  'eort',
  'is',
  'made',
  'to',
  'allow',
  'all',
  'student',
  'to',
  'fully',
  'experience',
  'life',
  'at',
  'the',
  'university'],
 ['students',
  'with',
  'the',
  'relevant',
  'supporng',
  'documentaon',
  'eg',
  'medical',
  'note',
  'andor',
  'report',
  'from',
  'educaonal',
  'psychologist',
  'are',
  'eligible',
  'to',
  'apply',
  'for',
  'appropriate',
  'support',
  'without',
  'compromising',
  'academic',
  'standard'],
 ['for',
  'the',
  'purpose',
  'of',
  'the',
  'equality',
  'act',
  'a',
  'person',
  'ha',
  'a',
  'disability',
  'if',
  'they',
  'have'

In [19]:
# replace the old comments with the new pre-processed comments
data = {'Sentence': postfin1}
p1 = pd.DataFrame(data)

In [20]:
p1

,Sentence
0,"[student, disability, policy, last, modied, on..."
1,"[students, with, the, relevant, supporng, docu..."
2,"[for, the, purpose, of, the, equality, act, a,..."
3,"[disability, in, this, context, can, include, ..."
4,"[typically, this, would, be, one, or, a, combi..."
...,...
114,"[note, student, what, should, the, scribe, do,..."
115,"[should, the, scribe, ask, you, to, repeat, it..."
116,"[note, student, how, will, the, scribe, indica..."
117,"[note, student, do, you, want, the, scribe, to..."


In [21]:
# set display options to show all columns and rows in the dataframe so i can quickly scan through
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)

# display the dataframe
print(p1)

                                              Sentence
0    [student, disability, policy, last, modied, on...
1    [students, with, the, relevant, supporng, docu...
2    [for, the, purpose, of, the, equality, act, a,...
3    [disability, in, this, context, can, include, ...
4    [typically, this, would, be, one, or, a, combi...
5    [rsi, mental, health, illness, condion, that, ...
6    [if, a, student, feel, that, they, have, a, co...
7    [the, university, is, commied, to, ensuring, r...
8    [the, university, us, the, term, disabled, stu...
9    [this, term, ha, been, adopted, to, reect, the...
10   [further, this, is, the, terminology, most, wi...
11   [however, it, is, acknowledged, that, some, st...
12   [students, are, responsible, for, informing, t...
13   [requests, for, strict, condenality, may, mean...
14   [where, a, student, informs, the, university, ...
15   [the, university, will, manage, the, process, ...
16   [students, are, responsible, for, informing, a...
17   [stud

In [22]:
# Doing so helped me notice that there are a bunch of empty lists, so we will remove those now
# find rows with empty lists in the 'Sentence' column
rows_with_empty_lists = p1[p1['Sentence'].apply(len) == 0]

# display rows with empty lists
print(rows_with_empty_lists)

Empty DataFrame
Columns: [Sentence]
Index: []


In [23]:
# remove rows with empty lists in the 'Sentence' column
p1 = p1[p1['Sentence'].apply(len) > 0]

# reset index
p1.reset_index(drop=True, inplace=True)

# display the modified DataFrame
print(p1)

                                              Sentence
0    [student, disability, policy, last, modied, on...
1    [students, with, the, relevant, supporng, docu...
2    [for, the, purpose, of, the, equality, act, a,...
3    [disability, in, this, context, can, include, ...
4    [typically, this, would, be, one, or, a, combi...
5    [rsi, mental, health, illness, condion, that, ...
6    [if, a, student, feel, that, they, have, a, co...
7    [the, university, is, commied, to, ensuring, r...
8    [the, university, us, the, term, disabled, stu...
9    [this, term, ha, been, adopted, to, reect, the...
10   [further, this, is, the, terminology, most, wi...
11   [however, it, is, acknowledged, that, some, st...
12   [students, are, responsible, for, informing, t...
13   [requests, for, strict, condenality, may, mean...
14   [where, a, student, informs, the, university, ...
15   [the, university, will, manage, the, process, ...
16   [students, are, responsible, for, informing, a...
17   [stud

In [24]:
# just to check it is still all good in this format too (can't be too careful)
p1

,Sentence
0,"[student, disability, policy, last, modied, on..."
1,"[students, with, the, relevant, supporng, docu..."
2,"[for, the, purpose, of, the, equality, act, a,..."
3,"[disability, in, this, context, can, include, ..."
4,"[typically, this, would, be, one, or, a, combi..."
5,"[rsi, mental, health, illness, condion, that, ..."
6,"[if, a, student, feel, that, they, have, a, co..."
7,"[the, university, is, commied, to, ensuring, r..."
8,"[the, university, us, the, term, disabled, stu..."
9,"[this, term, ha, been, adopted, to, reect, the..."


In [25]:
# save the dataset to be able to use in analysis

#export to excel file
p1.to_excel('p4.xlsx')

#export to csv file
p1.to_csv('p4.csv', index=False)